# Fine-tuning de BERT pour l'analyse de sentiment

**Auteur :** MUNIM Mehdi

**Date :** 2023-11-18

**Description :**

Ce notebook explore le fine-tuning de modèles BERT pré-entraînés pour la classification de sentiment sur des tweets.

**Objectif :**

L'objectif est de fine-tuner un modèle BERT pour prédire avec précision le sentiment (positif ou négatif) exprimé dans des tweets.

In [ ]:
#@title Imports

# Librairies pour la manipulation et l'analyse des données
import pandas as pd
import numpy as np

# Librairies pour le traitement du langage naturel (NLP)
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Librairies pour la visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Librairies pour le Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Librairies pour les embeddings
from gensim.models import Word2Vec, Doc2Vec

# Librairies pour la sauvegarde des données
import os
import pickle

# Librairies pour BERT
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoTokenizer, TFAutoModelForSequenceClassification

# Téléchargement des ressources NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## 1. Téléchargement et préparation des données

Dans cette section, nous allons télécharger les données, les charger dans un DataFrame pandas, puis les prétraiter et les tokenizer pour les utiliser avec les modèles BERT.

In [ ]:
#@title 1.1 Téléchargement des données

!wget "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip"
!unzip sentiment140.zip

--2024-11-19 11:32:56--  https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.105.242, 52.218.57.3, 52.92.1.112, ...
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.105.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84855679 (81M) [application/zip]
Saving to: ‘sentiment140.zip.2’

sentiment140.zip.2  100%[===================>]  80.92M  8.82MB/s    in 21s     

2024-11-19 11:33:17 (3.94 MB/s) - ‘sentiment140.zip.2’ saved [84855679/84855679]

Archive:  sentiment140.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: training.1600000.processed.noemoticon.csv  y



In [ ]:
#@title 1.2 Chargement et prétraitement des données

import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Charger les données du fichier CSV
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

# Fonction pour nettoyer le texte d'un tweet
def clean_tweet(tweet):
    # Supprimer les mentions, les hashtags et les liens
    tweet = re.sub(r"@[A-Za-z0-9_]+|#[A-Za-z0-9_]+|http\S+", "", tweet)
    # Supprimer les caractères spéciaux et la ponctuation
    tweet = re.sub(r"[^a-zA-Z ]", "", tweet)
    # Convertir le texte en minuscules
    tweet = tweet.lower()
    # Supprimer les mots vides
    stop_words = set(stopwords.words('english'))
    tweet = " ".join([word for word in tweet.split() if word not in stop_words])
    # Lemmatiser les mots
    lemmatizer = WordNetLemmatizer()
    tweet = " ".join([lemmatizer.lemmatize(word) for word in tweet.split()])
    return tweet

# Appliquer la fonction de nettoyage à la colonne des tweets
df[5] = df[5].apply(clean_tweet)

In [ ]:
#@title 1.2' Échantillonage

# Échantillonner 1% des données
df = df.sample(frac=0.01, random_state=42)

In [ ]:
#@title 1.3 Tokenization des données

from transformers import AutoTokenizer

# Charger le tokenizer BERT
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokeniser les tweets
X = tokenizer(
    df[5].tolist(),
    padding='max_length',
    max_length=50,
    truncation=True,
    return_tensors="tf"
)
y = df[0].map({0: 0, 4: 1})  # Convertir les labels en 0 et 1

In [12]:
#@title 1.4 Division en ensembles d'entraînement et de test

from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
(
    input_ids_train,
    input_ids_test,
    attention_mask_train,
    attention_mask_test,
    token_type_ids_train,
    token_type_ids_test,
    labels_train,
    labels_test,
) = train_test_split(
    X["input_ids"],
    X["attention_mask"],
    X["token_type_ids"],
    y,
    test_size=0.2,
    stratify=y,
    random_state=42,
)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([ 9094,  4284,   622, ..., 11908, 11743, 11335])

## 2. Fine-tuning du modèle BERT

Dans cette section, nous allons charger un modèle BERT pré-entraîné, définir les hyperparamètres d'entraînement, puis fine-tuner le modèle sur les données d'entraînement.


In [ ]:
#@title 2.1 Chargement du modèle BERT

from transformers import TFAutoModelForSequenceClassification

# Charger le modèle BERT pré-entraîné
model_name = "bert-base-uncased"  # Nom du modèle BERT à utiliser
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
#@title 2.2 Définition des hyperparamètres

# Définir les hyperparamètres du modèle
learning_rate = 2e-5
epochs = 3
batch_size = 8  # Réduire la taille des batchs pour économiser de la RAM

In [ ]:
#@title 2.3 Entraînement du modèle

# Compiler le modèle
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Entraîner le modèle
history = model.fit(
    x=[input_ids_train, attention_mask_train, token_type_ids_train],
    y=labels_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
)

In [ ]:
#@title 2.4 Évaluation du modèle

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)

# Fonction pour afficher les métriques d'évaluation
def evaluer_modele(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

    # Afficher la matrice de confusion
    plt.figure(figsize=(5, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title("Matrice de confusion")
    plt.ylabel("Vraie classe")
    plt.xlabel("Classe prédite")
    plt.show()

# Prédictions du modèle
y_pred = model.predict(
    [input_ids_test, attention_mask_test, token_type_ids_test]
).logits
y_pred = np.argmax(y_pred, axis=1)

# Évaluation du modèle
evaluer_modele(labels_test, y_pred)

## 3. Analyse des résultats

Dans cette section, nous allons analyser les performances du modèle fine-tuné en utilisant différentes métriques et visualisations.

In [ ]:
#@title 3.1 Affichage des métriques

# Prédictions du modèle
y_pred = model.predict(
    [input_ids_test, attention_mask_test, token_type_ids_test]
).logits
y_pred = np.argmax(y_pred, axis=1)

# Évaluation du modèle
evaluer_modele(labels_test, y_pred)

In [ ]:
#@title 3.2 Analyse des erreurs

# Afficher quelques exemples de tweets mal classés
for i in range(len(labels_test)):
    if labels_test[i] != y_pred[i]:
        print(f"Tweet : {df[5][i]}")
        print(f"Vraie classe : {labels_test[i]}")
        print(f"Classe prédite : {y_pred[i]}")
        print("-" * 20)

In [ ]:
#@title 3.3 Visualisation des résultats

# Afficher la courbe d'apprentissage
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy du modèle')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Entraînement', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss du modèle')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Entraînement', 'Validation'], loc='upper left')
plt.show()

## 4. Sauvegarde du modèle fine-tuné

Dans cette section, nous allons sauvegarder le modèle BERT fine-tuné pour une utilisation ultérieure.

In [ ]:
#@title 4.1 Sauvegarde du modèle fine-tuné

# Sauvegarder le modèle fine-tuné
model.save_pretrained("mon_modele_bert_finetuned")